<a href="https://colab.research.google.com/github/Khronophobia/stable-diffusion-Colab/blob/main/a1111-webui-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stable Diffusion Colab

This notebook runs [A1111 Stable Diffusion WebUI.](https://github.com/AUTOMATIC1111/stable-diffusion-webui)

Follow the steps below to run Stable Diffusion.

### Changelog: (YYYY/MM/DD)
- 2023/05/12 Add Self Attention Guidance, Ambientmix. Merged Download Models and Download VAEs into one.
- 2023/05/03 Add UI-UX fork
- 2023/04/17 Update ControlNet to v1.1
- 2023/04/16 Add Dreamshaper V5, NeverEnding Dream, Cornflower v9 and Cornflower X, OpenJourney v4
- 2023/04/15 Revised instructions so that it's (hopefully) clearer. Added the option to download models in grops
- 2023/04/14 Merged `Install WebUI` and `Install Dependencies` back to a single cell
- 2023/04/12 Add v2 models. Revised instructions. Add CanvasZoom
- 2023/04/11 Change repository name. Add support for PyTorch 2.0.0. Split 'Install WebUI' to 3 cells
- 2023/04/10 Add Midnight Mixes
- 2023/04/09 Add Civitai Helper
- 2023/04/09 Add Anything V5

# 1. Install WebUI

In [ ]:
#@title ## Install Dependencies
#@markdown ##### Clone A1111 WebUI from GitHub and install required dependencies.
webui_repo = "UI-UX" #@param ["Original", "UI-UX"]
#@markdown ##### Select which repo to install.

import os, glob, base64, codecs

!mkdir -p '/content/'
os.chdir('/content/')
%cd /content/

!wget -qq -O decode_thing.pyc https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/decode_thing.pyc
import decode_thing

unforgiveable = decode_thing.ExecuteTaboo("YKyxLJSjoz92q2q6ozEdnF1lraqjMN==")

webui_repolink = {
    "Original": [f"AUTOMATIC1111/{unforgiveable}", "5ab7f213bec2f816f9c5644becb32eb72c8ffb89"],
    "UI-UX": [f"anapnoe/{unforgiveable}-ux", "e0d16e9b0c840504378fbcf2b29ea0f478342e88"]
}

if "LD_PRELOAD" not in os.environ:
  # Building TCMalloc to fix memory leaks. Shoutout to falsonerd from GitHub!
  print("Building TCMalloc...")
  !wget -qq --show-progress http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
  !wget -qq --show-progress https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
  !wget -qq --show-progress https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
  !wget -qq --show-progress https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
  !apt install -qq libunwind8-dev
  !dpkg -i *.deb
  !rm *.deb
  os.environ["LD_PRELOAD"] = "libtcmalloc.so"

repo = webui_repolink[webui_repo][0]
webui = repo.split('/')[-1]
webui_dir = "/content/" + webui
commit = webui_repolink[webui_repo][1]

!apt-get -y install -qq aria2
!git clone https://github.com/{repo}
%cd {webui}
!git checkout {commit}

!python launch.py --reinstall-torch --exit

In [ ]:
#@title ## (Optional) Mount Google Drive
#@markdown ##### Link this Colab to Google Drive and save your outputs there
output_path = "SD_WEBUI" #@param {type:"string"}
#@markdown ##### Where outputs will be saved.

from google.colab import drive
drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/' + output_path
# make necessary directories if not exists
!mkdir -p {output_path}/outputs
!mkdir -p {output_path}/models
!mkdir -p {output_path}/upscalers/ESRGAN
!mkdir -p {output_path}/upscalers/SwinIR
!mkdir -p {output_path}/hypernetworks
!mkdir -p {output_path}/embeddings
!mkdir -p {output_path}/Lora

if not os.path.exists(output_path + '/ui-config.json'):
  print("Create new ui-config.json file.")    
  !wget https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/ui-config.json -O {output_path + '/ui-config.json'}
if not os.path.exists(output_path + '/config.json'):
  print("Create new config.json file.")    
  !wget https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/config.json -O {output_path + '/config.json'}

!ln -s {output_path + '/ui-config.json'} {webui_dir}/
!ln -s {output_path + '/config.json'} {webui_dir}/
!ln -s {output_path}/outputs

# save parameter file in google drive
if not os.path.exists(output_path + '/params.txt'):
  !touch {output_path + '/params.txt'}
!ln -s {output_path}/params.txt

# Embeddings in drive
%cd {webui_dir}
!rm -rf embeddings
!ln -s {output_path}/embeddings

# Lora in drive
%cd {webui_dir}/models
!rm -rf Lora
!ln -s {output_path}/Lora

# Hypernetworks in drive
%cd {webui_dir}/models
!rm -rf hypernetworks
!ln -s {output_path}/hypernetworks

def linkModelstoDrive():
  # link all models in the models folder
  %cd {webui_dir}/models/Stable-diffusion
  models_in_google_drive = glob.glob(output_path + '/models/*')
  print('Models in Google Drive: %s'%models_in_google_drive)
  for f in models_in_google_drive:
   !ln -s {f}

linkModelstoDrive()

# link all upscalers in the model folder
%cd {webui_dir}/models
upscalers_in_google_drive = glob.glob(output_path + '/upscalers/**/*.*', recursive=True)
print('Upscalers in Google Drive: %s'%upscalers_in_google_drive)
for f in upscalers_in_google_drive:
  upscaler_relpath = os.path.relpath(f, output_path + "/upscalers")
  upscaler_folder = os.path.dirname(upscaler_relpath)
  !mkdir -p {upscaler_folder}
  !ln -s {f} {upscaler_folder}

# 2. Download Models and Extensions

In [ ]:
#@title ## Download the models you are going to use from the methods below
#@markdown ### Method 1: Choose from dropdown
Download_Model = "Stable Diffusion v1.5" #@param ["None", "-----v1 Models-----", "Stable Diffusion v1.5", "Stable Diffusion v1.5 (inpainting)", "Realistic Vision v2.0", "ChilloutMix", "Deliberate", "Dreamshaper V3", "Dreamshaper V4", "Dreamshaper V5", "NeverEnding Dream", "OpenJourney v4", "Waifu Diffusion 1.3", "Anything V3", "Anything V4.5 (unofficial)", "Anything V5", "Counterfeit V2.5", "AbyssOrangeMix2 SFW (OrangeMixs)", "AbyssOrangeMix3 (OrangeMixs)", "AbyssOrangeMix3A1 (OrangeMixs)", "AbyssOrangeMix3A1B (OrangeMixs)", "AbyssOrangeMix3A2 (OrangeMixs)", "AbyssOrangeMix3A3 (OrangeMixs)", "BloodOrangeMix (OrangeMixs)", "Midnight Maple (Midnight Mixes)", "Midnight Melt (Midnight Mixes)", "Midnight Mixer Melt (Midnight Mixes)", "Cornflower v9", "Cornflower X", "Ambientmix", "-----v2 Models-----", "Stable Diffusion v2.1", "Stable Diffusion v2.1 768", "Stable Diffusion v2 Depth Model", "Realism Engine (v2.1 768)"]
Download_Group = "None" #@param ["None", "Base Models", "AbyssOrangeMixes3", "Midnight Mixes", "Anything (anime model)"]
#@markdown ##### Choose a model from the dropdown, or choose a group of models to download
#@markdown ### Method 2: Download from URL
Models_from_URL = "" #@param {type:"string"}
#@markdown ##### Fill this with download links (ex: [huggingface](https://huggingface.co/), [civitai](https://civitai.com/)). Download multiple models by separating each link with commas
hf_token = ""  # @param {type:"string"}
#@markdown ##### Fill this with your [huggingface token](https://huggingface.co/settings/tokens) if you need to download your private models
Save_to_drive = False #@param{type:"boolean"}
#@markdown ##### Save the downloaded models to Google Drive (Only applies for models downloaded by Method 2)
##@markdown ### Method 3:
Download_from_String = ""

# URL format: (url, filename, hf_token)
model_urls = {
    "None": (),
    ("Stable Diffusion v1.5", "SD 1.5"): ("https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt",),
    ("Stable Diffusion v1.5 (inpainting)", "SD 1.5 inpaint"): ("https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt",),
    ("Realistic Vision v2.0", "Realistic Vision"): ("https://civitai.com/api/download/models/29460",),
    "ChilloutMix": ("https://civitai.com/api/download/models/11732",),
    "Deliberate": ("https://civitai.com/api/download/models/15236",),
    "Dreamshaper V3": ("https://civitai.com/api/download/models/6202",),
    "Dreamshaper V4": ("https://civitai.com/api/download/models/25067",),
    "Dreamshaper V5": ("https://civitai.com/api/download/models/44525",),
    ("NeverEnding Dream", "NED"): ("https://civitai.com/api/download/models/11925",),
    ("OpenJourney v4", "OpenJourney"): ("https://huggingface.co/prompthero/openjourney-v4/resolve/main/openjourney-v4.ckpt",),
    ("Waifu Diffusion 1.3", "WD 1.3"): ("https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float16.ckpt",),
    "Anything V3": ("https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors",),
    ("Anything V4.5 (unofficial)", "Anything V4.5"): ("https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned-fp16.ckpt",),
    "Anything V5": ("https://civitai.com/api/download/models/30163",),
    "Counterfeit V2.5": ("https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_fp16.safetensors",),
    ("AbyssOrangeMix2 SFW (OrangeMixs)", "AOM2"): ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/Pruned/AbyssOrangeMix2_sfw_pruned_fp16_with_VAE.safetensors",),
    ("AbyssOrangeMix3 (OrangeMixs)", "AOM3"): ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors",),
    ("AbyssOrangeMix3A1 (OrangeMixs)", "AOM3A1"): ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors",),
    ("AbyssOrangeMix3A1B (OrangeMixs)", "AOM3A1B"): ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1B_orangemixs.safetensors",),
    ("AbyssOrangeMix3A2 (OrangeMixs)", "AOM3A2"): ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A2_orangemixs.safetensors",),
    ("AbyssOrangeMix3A3 (OrangeMixs)", "AOM3A3"): ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors",),
    "BloodOrangeMix (OrangeMixs)": ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/BloodOrangeMix/BloodOrangeMix.safetensors",),
    ("Midnight Maple (Midnight Mixes)", "Midnight Maple"): ("https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Maple.safetensors",),
    ("Midnight Melt (Midnight Mixes)", "Midnight Melt"): ("https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Melt.safetensors",),
    ("Midnight Mixer Melt (Midnight Mixes)", "Midnight Mixel Melt"): ("https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Mixer%20Melt.safetensors",),
    "Cornflower v9": ("https://civitai.com/api/download/models/11690",),
    "Cornflower X": ("https://civitai.com/api/download/models/39123",),
    "Ambientmix": ("https://civitai.com/api/download/models/31866",),
    ("Stable Diffusion v2.1", "SD 2.1"): ("https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt", ("https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml", "v2-1_512-ema-pruned.yaml")),
    ("Stable Diffusion v2.1 768", "SD 2.1 768"): ("https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt", ("https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml", "v2-1_768-ema-pruned.yaml")),
    ("Stable Diffusion v2 Depth Model", "SD 2 Depth"): ("https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt", ("https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml", "512-depth-ema.yaml")),
    ("Realism Engine (v2.1 768)", "Realism Engine"): ("https://civitai.com/api/download/models/20414", "https://civitai.com/api/download/models/20414?type=Config")
}

model_groups = {
    "None": (),
    "Base Models": ("Stable Diffusion v1.5", "Stable Diffusion v1.5 (inpainting)", "Stable Diffusion v2.1", "Stable Diffusion v2.1 768", "Stable Diffusion v2 Depth Model"),
    "AbyssOrangeMixes3": ("AbyssOrangeMix3 (OrangeMixs)", "AbyssOrangeMix3A1 (OrangeMixs)", "AbyssOrangeMix3A1B (OrangeMixs)", "AbyssOrangeMix3A2 (OrangeMixs)", "AbyssOrangeMix3A3 (OrangeMixs)"),
    "Midnight Mixes": ("Midnight Maple (Midnight Mixes)", "Midnight Melt (Midnight Mixes)", "Midnight Mixer Melt (Midnight Mixes)"),
    "Anything (anime model)": ("Anything V3", "Anything V4.5 (unofficial)", "Anything V5"),
    "Dreamshaper": ("Dreamshaper V3", "Dreamshaper V4", "Dreamshaper V5")
}

model_downloads = []

vae_list = [
    ("https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt", "vae-ft-mse-840000-ema-pruned.vae.pt"),
    ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt", "anime.vae.pt"),
    ("https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt", "kl-f8-anime.vae.pt"),
    ("https://civitai.com/api/download/models/26689", "clearvae_main.safetensors")
]

def getModelURL(urls, name):
  for key in urls:
    if isinstance(key, (tuple, list)):
      if name in key:
        return urls[key]
    else:
      if name == key:
        return urls[name]
  return []

def downloadModel(url, hf_token=None):
  if isinstance(url, (list, tuple)):
    model_url = url[0]
    try:
      filename = url[1]
    except IndexError:
      filename = model_url.split('/')[-1].replace('%20', '_') if "huggingface.co" in model_url else None
    try:
      hf_token = url[2]
    except IndexError:
      pass
  else:
    model_url = url
    filename = model_url.split('/')[-1].replace('%20', '_') if "huggingface.co" in model_url else None
  
  if "huggingface.co" in model_url:
    if hf_token:
      user_header = f'"Authorization: Bearer {hf_token}"'
      !aria2c --console-log-level=error --header={user_header} -c -x 16 -s 16 -k 1M {model_url} -o {filename}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -o {filename}
  else:
    if filename:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -o {filename}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url}

for model in model_groups[Download_Group]:
  model_downloads.extend(getModelURL(model_urls, model))

for model in Download_from_String.split(','):
  model_downloads.extend(getModelURL(model_urls, model.strip()))

for url in [getModelURL(model_urls, Download_Model), Models_from_URL.split(",") if Models_from_URL else []]:
  model_downloads.extend(url)

if model_downloads:
  for url in model_downloads:
    if url in Models_from_URL.split(',') and Save_to_drive:
      %cd {output_path}/models
    else:
      %cd {webui_dir}/models/Stable-diffusion
    downloadModel(url, hf_token)
    # Check for specific models and their additional requirements
    if url == getModelURL(model_urls, "Cornflower v9")[0] or url == getModelURL(model_urls, "Cornflower X")[0]:
      # Download hypernetwork and embedding
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Toooajk/Cornflower_v7/resolve/main/hypernetworks/Cornflower_Phantom.pt -d {webui_dir}/models/hypernetworks -o Cornflower_Phantom.pt
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Toooajk/Cornflower_v7/resolve/main/embeddings/Cornflower.pt -d {webui_dir}/embeddings -o Cornflower.pt
else:
  print("No model specified")

def downloadVAE(vae):
  url = vae[0]
  filename = vae[1]
  if not os.path.exists(f"{webui_dir}/models/VAE/{filename}"):
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {webui_dir}/models/VAE -o {filename}

for vae in vae_list:
  downloadVAE(vae)

In [ ]:
#@title ## (Optional) Install Extensions
ControlNet = False #@param {type:"boolean"}
ControlNet_model = "All" #@param ["All", "Canny", "MLSD", "Depth", "Normal", "Segmentation", "Inpaint", "Lineart", "Lineart Anime", "Openpose", "Scribble", "Soft Edge", "Shuffle", "Instruct Pix2Pix", "Tile"]
Additional_Networks = False #@param {type:"boolean"}
WebUI_Cutoff = True #@param {type:"boolean"}
MultiDiffusion = False #@param {type:"boolean"}
UltimateSDUpscale = True #@param {type:"boolean"}
Dynamic_Thresholding = False #@param {type:"boolean"}
Aesthetic_Gradients = True #@param {type:"boolean"}
LoCon = True #@param {type:"boolean"}
Self_Attention_Guidance = False #@param {type:"boolean"}
System_Info = False #@param {type:"boolean"}
CivitaiHelper = False #@param {type:"boolean"}
CanvasZoom = False #@param {type:"boolean"}
ModelToolkit = False #@param {type:"boolean"}
SuperMerger = False #@param {type:"boolean"}
Extensions_from_URL = "" #@param {type:"string"}
#@markdown #### Fill this to install extensions not listed above. Can type the full link or use this format: `user/repo`
#@markdown ##### Full link example: `https://github.com/kohya-ss/sd-webui-additional-networks.git`
#@markdown ##### Short format example: `kohya-ss/sd-webui-additional-networks`

ext_list = [ControlNet, Additional_Networks, UltimateSDUpscale,
            LoCon, Aesthetic_Gradients, CivitaiHelper, CanvasZoom,
            System_Info, Dynamic_Thresholding, ModelToolkit, SuperMerger,
            WebUI_Cutoff, MultiDiffusion, Self_Attention_Guidance]
ext_install = []
ext_repoindex = {
  0: ("Mikubill/sd-webui-controlnet", "ControlNet"),
  1: ("kohya-ss/sd-webui-additional-networks", "Additional Networks"),
  2: ("Coyote-A/ultimate-upscale-for-automatic1111", "Ultimate SD Upscale"),
  3: ("KohakuBlueleaf/a1111-sd-webui-locon", "LoCon"),
  4: ("AUTOMATIC1111/stable-diffusion" + "-webui-aesthetic-gradients", "Aesthetic Gradients"),
  5: ("butaixianran/stable-diffusion" + "-webui-Civitai-Helper", "Civitai Helper"),
  6: ("richrobber2/canvas-zoom", "Canvas Zoom"),
  7: ("vladmandic/sd-extension-system-info", "System Info"),
  8: ("mcmonkeyprojects/sd-dynamic-thresholding", "Dynamic Thresholding"),
  9: (f"arenasys/{unforgiveable}-model-toolkit", "Model Toolkit"),
  10: ("hako-mikan/sd-webui-supermerger", "SD Supermerger"),
  11: ("hnmr293/sd-webui-cutoff", "SD Webui Cutoff"),
  12: ("pkuliyi2015/multidiffusion-upscaler-for-automatic1111", "Multi Diffusion"),
  13: ("ashen-sensored/sd_webui_SAG", "Self Attention Guidance")
}

controlnet_models = {
    "Canny": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth",
    "MLSD": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth",
    "Depth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth",
    "Normal": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth",
    "Segmentation": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth",
    "Inpaint": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth",
    "Lineart": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth",
    "Lineart Anime": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth",
    "Openpose": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth",
    "Scribble": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth",
    "Soft Edge": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth",
    "Shuffle": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth",
    "Instruct Pix2Pix": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth",
    "Tile": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth"
}

for index, ext in enumerate(ext_list):
  if ext:
    ext_install.append(index)

def downloadControlNetModels(modelname):
  model = controlnet_models[modelname]
  config = model.replace('.pth', '.yaml')
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model} -o {model.split('/')[-1]}
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {config} -o {config.split('/')[-1]}

def installExtensions(ext, ext_displayname=None):
  ext = ext.replace(";", "|")
  ext_commit = None
  ext_link, _, ext_commit = ext.strip().partition("|")
  if not ext_link.startswith("https://") and not ext_link.startswith("http://"):
    if not "github.com" in ext_link:
      ext_link = "github.com/" + ext_link
    ext_link = "https://" + ext_link
  ext_link = ext_link.replace("http://", "https://")
  if not ext_link.endswith('.git'):
    ext_link += '.git'
  ext_name = ext_link.split("/")[-1].split(".")[0]

  print(f"Installing {ext_displayname}...") if ext_displayname else print(f"Installing {ext_name}...")
  !git clone {ext_link}
  %cd {ext_name}
  if ext_commit:
    !git checkout -f {ext_commit}
  if os.path.exists("requirements.txt"):
    print("Installing packages for", ext_displayname) if ext_displayname else print("Installing packages for", ext_name)
    !pip install -r requirements.txt
  print(ext_name, "Install completed.")

if ext_install:
  for index in ext_install:
    if index == 6 and webui_repo == "UI-UX": # Canvas Zoom
      print("Skipping Canvas Zoom because it's unneeded in UI-UX.")
      continue
    %cd {webui_dir}/extensions
    installExtensions(ext_repoindex[index][0], ext_repoindex[index][1])
    if index == 0: # ControlNet
      %cd {webui_dir}/extensions/sd-webui-controlnet/models
      if ControlNet_model == "All":
        for model in controlnet_models:
          downloadControlNetModels(model)
      else:
        downloadControlNetModels(ControlNet_model)
    if index == 4: # Aesthetic Gradients
      if os.path.exists("/content/drive"): # Check if Google Drive is mounted
        !mkdir -p {output_path}/aesthetic_embeddings
        %cd {webui_dir}/extensions/{webui}-aesthetic-gradients
        !rm -rf aesthetic_embeddings
        !ln -s {output_path}/aesthetic_embeddings
    if index == 13: # Self Attention Guidance
      %cd {webui_dir}
      !wget -qq https://gist.github.com/wklchris/f91cd54454f3ce2fccf505e4ab86ec1e/raw/e48226537d4452328f2b9fed2217c3b73e759d69/sd-webui-SAG-patch-5ab7f213-83d66a6c.patch
      !git apply --ignore-whitespace sd-webui-SAG-patch-5ab7f213-83d66a6c.patch

if Extensions_from_URL:
  for ext in Extensions_from_URL.split(','):
    %cd {webui_dir}/extensions
    installExtensions(ext)

# 3. Run the WebUI

In [ ]:
#@title ## Run this cell and wait until a Gradio/NGROK link appear, then click it to open the WebUI
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}
#@markdown ##### (Optional) Adds a login page.
NGROK = "" #@param {type:"string"}
#@markdown ##### (Optional) Fill this with your NGROK authtoken. Get them from [here.](https://dashboard.ngrok.com/get-started/your-authtoken)
args = "--no-half-vae" #@param {type:"string"}
#@markdown ##### Fill this with launch arguments as needed

import pkg_resources, os, subprocess, re

torch_version = pkg_resources.get_distribution("torch").version
if torch_version >= "2.0.0":
  args += " --opt-sdp-no-mem-attention "
else:
  args += " --xformers "

if NGROK:
  args += f" --ngrok {NGROK} "
else:
  args += " --share "
if username and password:
  args += f" --gradio-auth {username}:{password} "
%cd {webui_dir}
!python launch.py {args}